 ## Preliminaries

In [ ]:
#%% [code]

# Includes


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import numpy as np
import arviz as az
import pytensor.tensor as at



np.set_printoptions(suppress=True,precision=3)

In [ ]:
#%% [code]

# Load data

from load_data import load_data_for_experiments, load_data_for_experiment

experiment_version = "V1.1_pilot"
data = load_data_for_experiment(experiment_version)

#experiment_versions = ["V0.3_pilot", "V1.0_pilot", "V1.1_pilot"]
#data = load_data_for_experiments(experiment_versions)

counts = data['counts']

In [ ]:
#%% [code]

# Prepare data

# SYNTHETIC DATA for model recoverability
synthetic_hybrid_counts = np.tile(np.array([10, 5, 5, 1, 1, 4]), (30, 1))

synthetic_mixture_counts = np.vstack([
    np.tile(np.array([20, 1, 1, 1, 1, 4]), (15, 1)),
    np.tile(np.array([2, 10, 10, 1, 1, 4]), (15, 1))
])

synthetic_mixture_3_counts = np.vstack([
    np.tile(np.array([20, 1, 1, 1, 1, 4]), (10, 1)),
    np.tile(np.array([2, 10, 10, 1, 1, 4]), (10, 1)),
    np.tile(np.array([1, 1, 1, 1, 20, 4]), (10, 1))
])

synthetic_hybrid_2_counts = np.tile(np.array([10, 5, 5, 1, 10, 4]), (30, 1))

synthetic_mixture_4_counts = np.vstack([
    np.tile(np.array([10, 5, 5, 1, 1, 4]), (15, 1)),
    np.tile(np.array([1, 1, 1, 1, 20, 4]), (15, 1))
])
#counts = synthetic_mixture_3_counts

S, K = counts.shape
N = counts.sum(axis=1)

In [ ]:
#%% [code]

# Hybrid model: GPI zero + Policy reuse cued

with pm.Model() as hybrid_model:
    # positive raw weights for the tied pattern [n, m, m, e, e, 4e]
    u_e = pm.Uniform("u_e", lower=0.0, upper=1/9)
    u_n = pm.Uniform("u_n", lower=u_e, upper=1.0)
    u_m = pm.Uniform("u_m", lower=u_e, upper=1.0)

    # total concentration (how similar subjects are)
    c = pm.LogNormal("c", mu=0.0, sigma=1.5)

    # construct tied base proportions
    theta_raw = at.stack([u_n, u_m, u_m, u_e, u_e, 4*u_e])
    theta = theta_raw / theta_raw.sum()

    # Dirichlet parameters
    alpha = c * theta

    # vectorized Dirichlet–Multinomial over subjects
    pm.DirichletMultinomial("x", a=alpha, n=N, shape=(S, K), observed=counts)

    hybrid_trace = pm.sample(10000, tune=10000, target_accept=0.95, chains=8, idata_kwargs={"log_likelihood": True}, return_inferencedata=True)
    # Check convergence
    print("Hybrid model convergence diagnostics:")
    print(az.summary(hybrid_trace))
    # predictive accuracy (subject-level pointwise log-lik is handled internally)
    loo_hybrid = az.loo(hybrid_trace)     # or az.waic(idata)

Initializing NUTS using jitter+adapt_diag...
/home/momchil.tomov/miniconda3/envs/analysis/lib/python3.12/site-packages/pytensor/link/c/cmodule.py:2986: UserWarning: PyTensor could not link to a BLAS installation. Operations that might benefit from BLAS will be severely degraded.
This usually happens when PyTensor is installed via pip. We recommend it be installed via conda/mamba/pixi instead.
Alternatively, you can use an experimental backend such as Numba or JAX that perform their own BLAS optimizations, by setting `pytensor.config.mode == 'NUMBA'` or passing `mode='NUMBA'` when compiling a PyTensor function.
For more options and details see https://pytensor.readthedocs.io/en/latest/troubleshooting.html#how-do-i-configure-test-my-blas-library
  warnings.warn(
Multiprocess sampling (8 chains in 4 jobs)
NUTS: [u_e, u_n, u_m, c]


Output()

Sampling 8 chains for 10_000 tune and 10_000 draw iterations (80_000 + 80_000 draws total) took 131 seconds.


Hybrid model convergence diagnostics:
      mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
u_e  0.084  0.021   0.045    0.111      0.000    0.000   22977.0   28278.0   
u_n  0.241  0.075   0.094    0.376      0.000    0.000   26535.0   25412.0   
u_m  0.148  0.044   0.061    0.229      0.000    0.000   26017.0   25828.0   
c    7.983  1.365   5.534   10.555      0.007    0.006   41919.0   45989.0   

     r_hat  
u_e    1.0  
u_n    1.0  
u_m    1.0  
c      1.0  


In [ ]:
#%% [code]

# Mixture:
#   1) GPI zero
#   2) Policy reuse cued

with pm.Model() as mixture_model:
    # mixture weight
    w = pm.Dirichlet('w', a=np.ones(2))
    # raw weights (positive)
    u_e1  = pm.Uniform('u_e1', lower=0.0, upper=1/9)
    u_e2  = pm.Uniform('u_e2', lower=0.0, upper=1/9)
    u_n   = pm.Uniform('u_n', lower=u_e1, upper=1.0)
    u_m   = pm.Uniform('u_m', lower=u_e2, upper=1.0)

    # concentration
    c1 = pm.LogNormal('c1', 0.0, 1.5)
    c2 = pm.LogNormal('c2', 0.0, 1.5)

    # component base measures
    theta1_raw = pm.math.stack([u_n, u_e1, u_e1, u_e1, u_e1, 4*u_e1])
    theta2_raw = pm.math.stack([u_e2, u_m,  u_m,  u_e2, u_e2, 4*u_e2])
    theta1 = theta1_raw / pm.math.sum(theta1_raw)
    theta2 = theta2_raw / pm.math.sum(theta2_raw)

    alpha1 = c1 * theta1
    alpha2 = c2 * theta2

    # subject-level mixture likelihood (marginal over z)
    # PyMC has DirichletMultinomial: pm.DirichletMultinomial
    like1 = pm.DirichletMultinomial.dist(a=alpha1, n=counts.sum(axis=1))
    like2 = pm.DirichletMultinomial.dist(a=alpha2, n=counts.sum(axis=1))

    # mixture across subjects
    pm.Mixture('x', w, comp_dists=[like1, like2], observed=counts)

    mixture_trace = pm.sample(10000, tune=10000, target_accept=0.95, chains=8, idata_kwargs={"log_likelihood": True}, return_inferencedata=True)
    # Check convergence
    print("Mixture model convergence diagnostics:")
    print(az.summary(mixture_trace))
    loo_mixture = az.loo(mixture_trace)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (8 chains in 4 jobs)
NUTS: [w, u_e1, u_e2, u_n, u_m, c1, c2]


Output()

Sampling 8 chains for 10_000 tune and 10_000 draw iterations (80_000 + 80_000 draws total) took 331 seconds.


Mixture model convergence diagnostics:
        mean      sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
w[0]   0.832   0.082   0.681    0.969      0.000    0.000   61389.0   46101.0   
w[1]   0.168   0.082   0.031    0.319      0.000    0.000   61389.0   46101.0   
u_e1   0.075   0.026   0.028    0.111      0.000    0.000   28995.0   29633.0   
u_e2   0.065   0.027   0.021    0.111      0.000    0.000   48358.0   39400.0   
u_n    0.213   0.083   0.052    0.358      0.000    0.000   32590.0   28546.0   
u_m    0.572   0.241   0.186    1.000      0.001    0.001   44282.0   38287.0   
c1    11.151   2.298   7.189   15.507      0.009    0.011   67080.0   51189.0   
c2    15.340  15.217   1.389   35.264      0.074    0.637   74378.0   46271.0   

      r_hat  
w[0]    1.0  
w[1]    1.0  
u_e1    1.0  
u_e2    1.0  
u_n     1.0  
u_m     1.0  
c1      1.0  
c2      1.0  


In [ ]:
#%% [code]

# Mixture:
#   1) GPI zero
#   2) Policy reuse cued
#   3) Model-based / GPI

with pm.Model() as mixture_model_3:
    # mixture weight
    w = pm.Dirichlet('w', a=np.ones(3))
    
    # raw weights (positive)
    u_e1  = pm.Uniform('u_e1', lower=0.0, upper=1/9)
    u_e2  = pm.Uniform('u_e2', lower=0.0, upper=1/9)
    u_e3  = pm.Uniform('u_e3', lower=0.0, upper=1/9)
    u_n   = pm.Uniform('u_n', lower=u_e1, upper=1.0)
    u_m   = pm.Uniform('u_m', lower=u_e2, upper=1.0)
    u_o   = pm.Uniform('u_o', lower=u_e3, upper=1.0)

    # concentration
    c1 = pm.LogNormal('c1', 0.0, 1.5)
    c2 = pm.LogNormal('c2', 0.0, 1.5)
    c3 = pm.LogNormal('c3', 0.0, 1.5)

    # component base measures
    theta1_raw = pm.math.stack([u_n, u_e1, u_e1, u_e1, u_e1, 4*u_e1])
    theta2_raw = pm.math.stack([u_e2, u_m,  u_m,  u_e2, u_e2, 4*u_e2])
    theta3_raw = pm.math.stack([u_e3, u_e3,  u_e3,  u_e3, u_o, 4*u_e3])
    theta1 = theta1_raw / pm.math.sum(theta1_raw)
    theta2 = theta2_raw / pm.math.sum(theta2_raw)
    theta3 = theta3_raw / pm.math.sum(theta3_raw)
    
    alpha1 = c1 * theta1
    alpha2 = c2 * theta2
    alpha3 = c3 * theta3

    # subject-level mixture likelihood (marginal over z)
    # PyMC has DirichletMultinomial: pm.DirichletMultinomial
    like1 = pm.DirichletMultinomial.dist(a=alpha1, n=counts.sum(axis=1))
    like2 = pm.DirichletMultinomial.dist(a=alpha2, n=counts.sum(axis=1))
    like3 = pm.DirichletMultinomial.dist(a=alpha3, n=counts.sum(axis=1))

    # mixture across subjects
    pm.Mixture('x', w, comp_dists=[like1, like2, like3], observed=counts)

    mixture_trace_3 = pm.sample(10000, tune=10000, target_accept=0.95, chains=8, idata_kwargs={"log_likelihood": True}, return_inferencedata=True)
    # Check convergence
    print("Mixture model 3 convergence diagnostics:")
    print(az.summary(mixture_trace_3))
    loo_mixture_3 = az.loo(mixture_trace_3)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (8 chains in 4 jobs)
NUTS: [w, u_e1, u_e2, u_e3, u_n, u_m, u_o, c1, c2, c3]


Output()

Sampling 8 chains for 10_000 tune and 10_000 draw iterations (80_000 + 80_000 draws total) took 419 seconds.


Mixture model 3 convergence diagnostics:
        mean      sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
w[0]   0.634   0.184   0.295    0.925      0.001    0.001   19031.0   45605.0   
w[1]   0.156   0.077   0.028    0.298      0.000    0.000   82254.0   51215.0   
w[2]   0.210   0.179   0.000    0.531      0.001    0.001   15376.0   29418.0   
u_e1   0.075   0.026   0.028    0.111      0.000    0.000   52217.0   44682.0   
u_e2   0.064   0.027   0.021    0.111      0.000    0.000   49015.0   37453.0   
u_e3   0.069   0.029   0.019    0.111      0.000    0.000   56662.0   45460.0   
u_n    0.259   0.128   0.031    0.487      0.001    0.001   35824.0   40051.0   
u_m    0.586   0.240   0.196    1.000      0.001    0.001   46286.0   39562.0   
u_o    0.347   0.243   0.021    0.856      0.002    0.001   26385.0   34858.0   
c1    12.796   4.448   6.244   20.376      0.023    0.118   56204.0   37474.0   
c2    16.103  15.883   1.382   37.431      0.074    0.497   70494.0 

In [ ]:
#%% [code]

# Hybrid model 2: GPI zero + Policy reuse cued + MB/GPI

with pm.Model() as hybrid_model_2:
    # positive raw weights for the tied pattern [n, m, m, e, o, 4e]
    u_e = pm.Uniform("u_e", lower=0.0, upper=1/9)
    u_n = pm.Uniform("u_n", lower=u_e, upper=1.0)
    u_m = pm.Uniform("u_m", lower=u_e, upper=1.0)
    u_o = pm.Uniform("u_o", lower=u_e, upper=1.0)

    # total concentration (how similar subjects are)
    c = pm.LogNormal("c", mu=0.0, sigma=1.5)

    # construct tied base proportions
    theta_raw = at.stack([u_n, u_m, u_m, u_e, u_o, 4*u_e])
    theta = theta_raw / theta_raw.sum()

    # Dirichlet parameters
    alpha = c * theta

    # vectorized Dirichlet–Multinomial over subjects
    pm.DirichletMultinomial("x", a=alpha, n=N, shape=(S, K), observed=counts)

    hybrid_trace_2 = pm.sample(10000, tune=10000, target_accept=0.95, chains=8, idata_kwargs={"log_likelihood": True}, return_inferencedata=True)
    # Check convergence
    print("Hybrid model 2 convergence diagnostics:")
    print(az.summary(hybrid_trace_2))
    # predictive accuracy (subject-level pointwise log-lik is handled internally)
    loo_hybrid_2 = az.loo(hybrid_trace_2)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (8 chains in 4 jobs)
NUTS: [u_e, u_n, u_m, u_o, c]


Output()

Sampling 8 chains for 10_000 tune and 10_000 draw iterations (80_000 + 80_000 draws total) took 106 seconds.


Hybrid model 2 convergence diagnostics:
      mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
u_e  0.090  0.018   0.057    0.111      0.000    0.000   21918.0   28208.0   
u_n  0.339  0.093   0.160    0.515      0.001    0.000   24906.0   24967.0   
u_m  0.207  0.055   0.103    0.313      0.000    0.000   24549.0   24763.0   
u_o  0.282  0.079   0.133    0.433      0.000    0.000   25039.0   25115.0   
c    8.739  1.587   5.971   11.801      0.008    0.007   45467.0   46717.0   

     r_hat  
u_e    1.0  
u_n    1.0  
u_m    1.0  
u_o    1.0  
c      1.0  


In [ ]:
#%% [code]

# Mixture 4:
#   1) GPI zero + Policy reuse cued
#   2) MB/GPI

with pm.Model() as mixture_model_4:
    # mixture weight
    w = pm.Dirichlet('w', a=np.ones(2))
    
    # raw weights (positive)
    u_e1  = pm.Uniform('u_e1', lower=0.0, upper=1/9)
    u_e2  = pm.Uniform('u_e2', lower=0.0, upper=1/9)
    u_n   = pm.Uniform('u_n', lower=u_e1, upper=1.0)
    u_m   = pm.Uniform('u_m', lower=u_e1, upper=1.0)
    u_o   = pm.Uniform('u_o', lower=u_e2, upper=1.0)

    # concentration
    c1 = pm.LogNormal('c1', 0.0, 1.5)
    c2 = pm.LogNormal('c2', 0.0, 1.5)

    # component base measures
    # Component 1: GPI zero + Policy reuse cued [n, m, m, e, e, 4e]
    theta1_raw = pm.math.stack([u_n, u_m, u_m, u_e1, u_e1, 4*u_e1])
    # Component 2: MB/GPI [e, e, e, e, o, 4e]
    theta2_raw = pm.math.stack([u_e2, u_e2, u_e2, u_e2, u_o, 4*u_e2])
    theta1 = theta1_raw / pm.math.sum(theta1_raw)
    theta2 = theta2_raw / pm.math.sum(theta2_raw)
    
    alpha1 = c1 * theta1
    alpha2 = c2 * theta2

    # subject-level mixture likelihood (marginal over z)
    # PyMC has DirichletMultinomial: pm.DirichletMultinomial
    like1 = pm.DirichletMultinomial.dist(a=alpha1, n=counts.sum(axis=1))
    like2 = pm.DirichletMultinomial.dist(a=alpha2, n=counts.sum(axis=1))

    # mixture across subjects
    pm.Mixture('x', w, comp_dists=[like1, like2], observed=counts)

    mixture_trace_4 = pm.sample(10000, tune=10000, target_accept=0.95, chains=8, idata_kwargs={"log_likelihood": True}, return_inferencedata=True)
    # Check convergence
    print("Mixture model 4 convergence diagnostics:")
    print(az.summary(mixture_trace_4))
    loo_mixture_4 = az.loo(mixture_trace_4)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (8 chains in 4 jobs)
NUTS: [w, u_e1, u_e2, u_n, u_m, u_o, c1, c2]


Output()

Sampling 8 chains for 10_000 tune and 10_000 draw iterations (80_000 + 80_000 draws total) took 331 seconds.


Mixture model 4 convergence diagnostics:
       mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
w[0]  0.724  0.261   0.209    1.000      0.003    0.002   10523.0   18010.0   
w[1]  0.276  0.261   0.000    0.791      0.003    0.002   10523.0   18010.0   
u_e1  0.084  0.021   0.044    0.111      0.000    0.000   37245.0   34947.0   
u_e2  0.070  0.028   0.020    0.111      0.000    0.000   49260.0   41848.0   
u_n   0.274  0.120   0.067    0.491      0.001    0.001   30109.0   27889.0   
u_m   0.212  0.159   0.024    0.540      0.002    0.002   15361.0   15785.0   
u_o   0.325  0.238   0.015    0.831      0.002    0.001   21070.0   31641.0   
c1    8.765  4.450   3.397   13.907      0.028    0.504   32542.0   25700.0   
c2    7.008  7.501   0.007   17.810      0.048    0.262   16960.0   32908.0   

      r_hat  
w[0]    1.0  
w[1]    1.0  
u_e1    1.0  
u_e2    1.0  
u_n     1.0  
u_m     1.0  
u_o     1.0  
c1      1.0  
c2      1.0  


/home/momchil.tomov/miniconda3/envs/analysis/lib/python3.12/site-packages/arviz/stats/stats.py:1057: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)


In [ ]:
#%% [code]

# Comprehensive convergence diagnostics

print("\n" + "="*80)
print("CONVERGENCE DIAGNOSTICS SUMMARY")
print("="*80)

traces = {
    "hybrid": hybrid_trace,
    "mixture": mixture_trace,
    "mixture_3": mixture_trace_3,
    "hybrid_2": hybrid_trace_2,
    "mixture_4": mixture_trace_4
}

for name, trace in traces.items():
    print(f"\n{name.upper()} MODEL:")
    print("-" * 80)
    # Get summary which includes R-hat and ESS
    summary = az.summary(trace)
    
    # Extract max R-hat and min ESS from summary
    if 'r_hat' in summary.columns:
        max_rhat = float(summary['r_hat'].max())
        print(f"Max R-hat: {max_rhat:.4f} {'✓' if max_rhat < 1.01 else '✗ WARNING: R-hat > 1.01'}")
    else:
        # Fallback: compute directly
        rhat = az.rhat(trace)
        max_rhat = float(rhat.max().to_numpy())
        print(f"Max R-hat: {max_rhat:.4f} {'✓' if max_rhat < 1.01 else '✗ WARNING: R-hat > 1.01'}")
    
    if 'ess_bulk' in summary.columns:
        min_ess = float(summary['ess_bulk'].min())
        print(f"Min ESS (bulk): {min_ess:.0f} {'✓' if min_ess > 400 else '✗ WARNING: ESS < 400'}")
    elif 'ess_mean' in summary.columns:
        min_ess = float(summary['ess_mean'].min())
        print(f"Min ESS (mean): {min_ess:.0f} {'✓' if min_ess > 400 else '✗ WARNING: ESS < 400'}")
    else:
        # Fallback: compute directly
        ess = az.ess(trace)
        min_ess = float(ess.min().to_numpy())
        print(f"Min ESS: {min_ess:.0f} {'✓' if min_ess > 400 else '✗ WARNING: ESS < 400'}")

print("\n" + "="*80 + "\n")


CONVERGENCE DIAGNOSTICS SUMMARY

HYBRID MODEL:
--------------------------------------------------------------------------------
Max R-hat: 1.0000 ✓
Min ESS (bulk): 22977 ✓

MIXTURE MODEL:
--------------------------------------------------------------------------------
Max R-hat: 1.0000 ✓
Min ESS (bulk): 28995 ✓

MIXTURE_3 MODEL:
--------------------------------------------------------------------------------
Max R-hat: 1.0000 ✓
Min ESS (bulk): 15376 ✓

HYBRID_2 MODEL:
--------------------------------------------------------------------------------
Max R-hat: 1.0000 ✓
Min ESS (bulk): 21918 ✓

MIXTURE_4 MODEL:
--------------------------------------------------------------------------------
Max R-hat: 1.0000 ✓
Min ESS (bulk): 10523 ✓




In [ ]:
#%% [code]

# Compare models

az.compare({
    "hybrid": hybrid_trace,
    "mixture": mixture_trace,
    "mixture_3": mixture_trace_3,
    "hybrid_2": hybrid_trace_2,
    "mixture_4": mixture_trace_4
})

/home/momchil.tomov/miniconda3/envs/analysis/lib/python3.12/site-packages/arviz/stats/stats.py:1057: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
hybrid_2,0,-311.662874,4.532164,0.000000,8.091327e-01,5.417645,0.000000,False,log
mixture,1,-318.372236,4.943443,6.709361,1.908673e-01,5.670672,7.258005,False,log
mixture_3,2,-318.835451,6.878545,7.172577,1.790888e-12,5.102855,6.939592,False,log
hybrid,3,-323.147045,3.702179,11.484170,1.810556e-12,5.239962,3.885712,False,log
mixture_4,4,-326.421326,9.240613,14.758452,1.061040e-11,4.522093,3.414154,False,log


In [ ]:
data['counts']

array([[13,  3,  0,  0,  2,  9],
       [ 6,  4,  2,  0,  5, 10],
       [ 2,  8,  4,  0,  9,  4],
       [ 4,  0,  3,  0,  8, 12],
       [12,  2,  2,  0,  6,  5],
       [ 8,  1,  0,  0,  7, 11],
       [ 3, 10, 10,  0,  3,  1],
       [ 1,  2,  8,  2,  6,  8],
       [ 8,  2,  1,  0,  6, 10],
       [14,  1,  4,  0,  1,  7],
       [ 0,  9,  9,  0,  3,  6],
       [ 5,  4,  4,  0,  3, 11],
       [ 0, 13, 12,  0,  0,  2],
       [ 7,  2,  2,  2,  6,  8],
       [ 2,  0,  8,  1,  6, 10],
       [ 4,  2,  3,  0, 12,  6],
       [ 2,  1,  3,  0,  6, 15],
       [ 7,  1,  4,  3,  4,  8],
       [ 5,  6,  1,  0,  3, 12],
       [ 5,  2,  1,  0,  6, 13],
       [ 4,  0,  4,  1,  2, 16],
       [ 8,  2,  8,  0,  0,  9],
       [ 7,  7,  2,  0,  3,  8],
       [ 4,  7,  7,  1,  1,  7],
       [12,  0,  0,  0, 12,  3],
       [ 2,  4,  7,  1,  2, 11],
       [12,  0,  1,  3,  4,  7],
       [16,  1,  1,  0,  5,  4]])

In [ ]:
hybrid_model_2

u_e ~ Uniform(0, 0.111)
u_n ~ Uniform(u_e, 1)
u_m ~ Uniform(u_e, 1)
u_o ~ Uniform(u_e, 1)
  c ~ LogNormal(0, 1.5)
  x ~ DirichletMultinomial(<constant>, f(c, u_o, u_e, u_m, u_n))

In [ ]:
with hybrid_model_2:
    posterior_predictive = pm.sample_posterior_predictive(hybrid_trace_2, predictions=True)
    # Access the samples
    posterior_samples = posterior_predictive.predictions["x"]



Sampling: [x]


Output()

In [ ]:
posterior_samples




<xarray.DataArray 'x' (chain: 8, draw: 10000, x_dim_0: 28, x_dim_1: 6)> Size: 108MB
array([[[[13,  1,  1,  2,  4,  6],
         [ 6,  0,  1,  1, 13,  6],
         [13,  1,  1,  2,  3,  7],
         ...,
         [ 4,  0,  0,  1, 12, 10],
         [15,  1,  5,  0,  0,  6],
         [ 4,  5,  2,  1,  3, 12]],

        [[ 2,  1,  2,  6,  0, 16],
         [ 5,  5,  0,  1,  8,  8],
         [ 4,  1,  1,  6,  1, 14],
         ...,
         [ 5,  6,  0,  3,  9,  4],
         [ 9,  3,  1,  3,  3,  8],
         [ 2,  3,  7,  1,  3, 11]],

        [[ 7,  2,  0,  6,  7,  5],
         [ 5,  4, 10,  0,  3,  5],
         [ 3,  5,  2,  0,  5, 12],
         ...,
...
         ...,
         [13,  1,  4,  0,  7,  2],
         [17,  3,  0,  1,  3,  3],
         [ 6,  2,  1,  2,  5, 11]],

        [[ 1,  4,  1,  0, 11, 10],
         [ 9,  1,  2,  1, 10,  4],
         [ 2,  7,  4,  0,  6,  8],
         ...,
         [ 5,  0,  2,  4,  1, 15],
         [ 2, 10,  0,  0, 13,  2],
         [ 7, 11,  2,  1,  5,  1]],

        [[10,  1,  4,  2,  2,  8],
         [ 5,  3,  8,  0,  2,  9],
         [14,  3,  2,  0,  0,  8],
         ...,
         [ 9,  2,  1,  0,  6,  9],
         [ 7,  0,  3,  0,  9,  8],
         [15,  1,  6,  0,  5,  0]]]], shape=(8, 10000, 28, 6))
Coordinates:
  * chain    (chain) int64 64B 0 1 2 3 4 5 6 7
  * draw     (draw) int64 80kB 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * x_dim_0  (x_dim_0) int64 224B 0 1 2 3 4 5 6 7 8 ... 20 21 22 23 24 25 26 27
  * x_dim_1  (x_dim_1) int64 48B 0 1 2 3 4 5

In [ ]:
u_e_samples = hybrid_trace_2.posterior["u_e"].values

In [ ]:
hybrid_trace_2.posterior["u_n"].values

array([[0.358, 0.263, 0.348, ..., 0.385, 0.253, 0.276],
       [0.338, 0.291, 0.284, ..., 0.35 , 0.278, 0.309],
       [0.291, 0.345, 0.35 , ..., 0.135, 0.171, 0.179],
       ...,
       [0.273, 0.481, 0.532, ..., 0.213, 0.356, 0.324],
       [0.392, 0.266, 0.347, ..., 0.338, 0.368, 0.255],
       [0.237, 0.292, 0.37 , ..., 0.448, 0.434, 0.453]], shape=(8, 10000))

In [ ]:
hybrid_trace_2.posterior["u_n"].mean().values

array(0.339)

In [ ]:
hybrid_trace_2.posterior["u_m"].mean().values

array(0.207)

In [ ]:
hybrid_trace_2.posterior["u_o"].mean().values

array(0.282)

In [ ]:
hybrid_trace_2.posterior["u_e"].mean().values

array(0.09)